# Deep learning for computer vision

Trong chapter này, ta sẽ học về convolutional neural networks ( hay còn gọi là convnets, hoặc CNN) - một dạng deep learning được sử dụng hầu hết trong mảng computer vision. Ta sẽ học cách áp dụng chúng cho bài toán classify ảnh, đặc biệt với data set nhỏ

Trong chương này, ta sẽ trả lời các câu hỏi sau:
- Convnets là gì ?
- Model convnets học như thế nào ?
- convolution và maxpooling là gì ?

Sau đó ta sẽ bao quát bài toán classify ảnh với data set nhỏ :
- Tự xây dựng và train 1 mạng convnets nhỏ 
- Sử dụng data augmentation (các phép biến đổi để tạo thêm data) giúp giảm bớt over-fitting
- Fine-tuning 1 model đã train trước đó

## 5.1 GIỚI THIỆU VỀ CONVNETS

Chúng ta sẽ tìm hiểu lý do tại sao convnets rất hay được sử dụng trong các bài toán computer vision. Nhưng đầu tiên ta sẽ thực hành với 1 ví dụ cơ bản về convnet : Dùng convnets để classify MNIST - dataset ta đã từng giải ở chapter 2 với Dense layer (acc ~97.8%).

Spoiler warning :Dù convnets model rất cơ bản, độ chính xác vẫn cao hơn Dense layer

Model convnets là một khối các layer Conv2D và MaxPooling2D. Quan trọng hơn, Convnet nhận vào 2D tensor với shape (image_height,image_width,image_channels) ( chưa kể batch ). Trong bài toán này, ta sẽ config input với size (28,28,1) : format của MNIST sample bằng cách pass input_shape=(28,28,1) vào layer đầu tiên

In [10]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Dễ thấy output của mỗi Conv2D và MaxPooling2D layer là 2D tensor với **shape (h,w,channels)**. w và h càng **giảm dần** khi **xuống các layer dưới**, trong khi **channels** ( số kênh ) được kiểm soát = **số đầu tiên pass vào Conv2D**

Bước tiếp theo ta sẽ feed tensor ( với shape (3,3,64)) vào 1 dense layer. Nhưng vì layer này chỉ nhận vector ( 1D ), ta sẽ sử dụng Flatten() layer

In [12]:
model.add(layers.Flatten()) # Model sẽ có shape (None,3*3*64) -> vector
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                36928     
__________

Bây giờ ta sẽ train convnet trên tập MNIST

In [14]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [15]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# reshape có thêm 1 thể hiện số kênh màu (1)
train_images = train_images.reshape((60000,28,28,1))
train_images = train_images.astype('float32') /255

test_images = test_images.reshape((10000,28,28,1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


In [16]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(train_images,train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 25s - loss: 0.1794 - acc: 0.9429    
Epoch 2/5
60000/60000 [==============================] - 26s - loss: 0.0476 - acc: 0.9857    
Epoch 3/5
60000/60000 [==============================] - 26s - loss: 0.0322 - acc: 0.9902    
Epoch 4/5
60000/60000 [==============================] - 26s - loss: 0.0245 - acc: 0.9928    
Epoch 5/5
60000/60000 [==============================] - 26s - loss: 0.0185 - acc: 0.9943    


In [17]:
# Evaluate model :
test_loss, test_acc = model.evaluate(test_images,test_labels)
test_acc

 9952/10000 [============================>.] - ETA: 0s

0.99039999999999995

### Quào, 99%,  network với nhiều Dense layer ở bài 2 chỉ được 97% 

Tại sao Convnet hoạt động quá tốt so với model sử dụng tất cả Dense layer ??? Để trả lời câu hỏi này, ta sẽ nghiên cứu Conv2D và MaxPooling2D